Jupyter notebook for doing the interview task for UMC:

**Analyse the drug side-effect reports (def:_narratives_)**

- look at differences between COVID and non-COVID cases
- what can we say about the text?
- suggestions in assignment:
    - first person? why?
    - redundant narrative? why?
- self investigations
    - text lengths and distributions wrt cases -- classifier. Histogram as feature for a Naive Bayes
        - Statistics
    - What features make it first person?
        - Statistics
    - redundancy detection:
        - duplication detection regexp within narrative
        - duplication of information from other columns and data files?
        - fluff content vs. entropy content
    - vocabulary differences: word frequency distributions TF-IDF 
    - content itself: keywords?
    - timing of narrative wrt symptoms?
    - look at all possibilities and their relevant narratives: FN, FP, TP, TN
    - *Sentiment analysis:* TextBlob, Vader, LLM?
    - *Topic models with LDA*


In [5]:
import pickle
import pandas as pd
from IPython.display import display
import numpy as np

data_filename = "2020VAERSDATA"
vax_filename = "2020VAERSVAX"
symptoms_filename = "2020VAERSSYMPTOMS"

Read in data and display relevant columns 

In [8]:
with open(data_filename+".pickle", 'rb') as fh:
    df_data, cols_data = pickle.load(fh)
    display(df_data[cols_data].head(5))
print("\n\n")
with open(vax_filename+".pickle", 'rb') as fh:
    df_vax, cols_vax = pickle.load(fh)
    display(df_vax[cols_vax].head(5))
print("\n\n")
with open(symptoms_filename+".pickle", 'rb') as fh:
    df_symptoms, cols_symptoms = pickle.load(fh)
    display(df_symptoms[cols_symptoms].head(5))


,VAERS_ID,SYMPTOM_TEXT
0,855017,Symptoms occurred almost immediately- aching j...
1,855018,"Extreme pain, muscle weakness in right arm so ..."
2,855019,SORENESS IN THE AREA. ITCHING AND RASH
3,855020,"sore arm, fever 101 , chills, and aching musc..."
4,855021,Patient presented to Clinic today 1/1/2020. Sh...


,VAERS_ID,VAX_NAME
0,855017,ZOSTER (SHINGRIX)
1,855018,VACCINE NOT SPECIFIED (OTHER)
2,855019,ZOSTER (SHINGRIX)
3,855020,ZOSTER (SHINGRIX)
4,855021,ZOSTER (SHINGRIX)


,VAERS_ID,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
0,855017,Arthralgia,Chills,Injection site pain,Pyrexia,NaN
1,855018,Chills,Fatigue,Hypertension,Hypoaesthesia,Injected limb mobility decreased
2,855018,Muscular weakness,Pain in extremity,Pyrexia,Tremor,Vertigo
3,855019,Pain,Pruritus,Rash,NaN,NaN
4,855020,Chills,Influenza like illness,Myalgia,Pain in extremity,Pyrexia


Check the total number of patient entries:

In [23]:
# Data Check
print(df_data[cols_data[0]].nunique())
print(df_vax[cols_data[0]].nunique())
print(df_symptoms[cols_data[0]].nunique())

print(set(df_data["VAERS_ID"]).symmetric_difference(set(df_vax["VAERS_ID"])))
print(set(df_data["VAERS_ID"]).symmetric_difference(set(df_symptoms["VAERS_ID"])))

49688
49688
49688
set()
set()


In [25]:
# temp
ids = [1123, 1345, 2345, 8876]
names = ["Karaboudjan", "covid-19", "co vid", "COVID-19" ]
dict4df = {
    "VAERS_ID": ids,
    "VAERS_VAX":names
}
df_toy = pd.DataFrame(dict4df)
display(df_toy)

,VAERS_ID,VAERS_VAX
0,1123,Karaboudjan
1,1345,covid-19
2,2345,co vid
3,8876,COVID-19


In [70]:
# get all rows where column is related to covid
query_keys = ["covid", "cov"]
df_covid = df_toy[df_toy["VAERS_VAX"].str.replace(" ", "").str.contains('|'.join(query_keys), case=False, regex=True)]
display(df_covid)

,VAERS_ID,VAERS_VAX
1,1345,covid-19
2,2345,co vid
3,8876,COVID-19
